# Data Anaysis Project part1:

I am using Vancouver rental ads on craigslist to analyze rental market. Later, I will create the class to be able to analyze dynamically. Also, I will start collecting the data and save them to the CSV, so I can analyze the trend of the rental market as well. 

I will follow what I am learning from Google Data Analytics course to complete this project.

* Ask
* Prepare
* Process
* Analyze
* Share
* Act



## Overview of the project:

Analyze housing rental market in Fraser Valley to find out what is hot to real estate investors in this area. Iam going to create the code to analyze dynamically so the investors can apply in their area for their own use.  


## ASK 
    
    Find out what is 'HOT' in the market for investor, so I can find a property for them. Find out supply and demand of the rental housing market and calculate appropreate purchase price. 

#### What are the questions to ask?
Supply side:
* How much is the market rental price?
* What type of the rental is most popular and least popular?
* How many ads per day? Number of supply 
* Is rental price rising or decreasing over time?
* What is the correlation between housing price and rental price?
* From those questions, what is my prediction of the rental market? 

Demand side:
* What kind of propety does people look for rent?
* What is the price range they are looking for?
* What are the purposes of renting. (Who is looking for rent)


#### Review Google Data Analysis course and create checklist:
* Ask effective questions
* Define the problems
* Using structured thinking
* Communicate with others



## Prepare  

#### Using Craigslist add to collect market data: (Supply side)
Scrape craigslist rental housing pages and store in csv file. 

<a href='https://github.com/rileypredum/East-Bay-Housing-Web-Scrape/blob/master/EB_Apt_Prices_Final.ipynb'>Code: I will use this code and modify for my situation.  </a> <br>
<a href='https://towardsdatascience.com/web-scraping-craigslist-a-complete-tutorial-c41cea4f4981'>Explaning</a>


#### Review from Google Data Analysis course:
* Understand how data is generated and collect. 
    * The data is collected from Craigslist ads. 
* Identify and use different data formats, types, and structures. 
* Make sure data is unbiased and credible. 
    * The data is live advertisement, so it is great to see today's market price. However, craigslist doesn't seems today's most favorite site to advertize so it might not accurately show the reality. 
* Organize and protect data. 

In [1]:
# Import libralis
from requests import get
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import numpy as np

from time import sleep
import re
from random import randint
from warnings import warn
from time import time
from IPython.display import clear_output

In [2]:
class Url:
    def __init__(self, city_name, sub_city=None, num_post=0):
        self.city_name = city_name
        self.sub_city = sub_city
        self.num_post = num_post
        self.url = self.get_url()
        
    def get_url(self):
        if self.city_name == 'Abbotsford':
            url = f'https://{self.city_name.lower()}.craigslist.org/search/apa?s={self.num_post}sort=date&bundleDuplicates=1&min_price=&max_price=&availabilityMode=0&sale_date=all+dates'
        else:
            url = f'https://{self.city_name.lower()}.craigslist.org/search/{self.sub_city}/apa?s={self.num_post}sort=date&bundleDuplicates=1&min_price=&max_price=&availabilityMode=0&sale_date=all+dates'
        return url 

In [3]:
# Vanvouver rental housing
url = Url("Vancouver", 'van')
url_ = url.url

response = get(url_)


soup = BeautifulSoup(response.text, 'html.parser')

posts = soup.find_all('li', class_='result-row')
print(type(posts)) # to double check that I got resultSet
print(len(posts)) # to double check I got 120 elements

<class 'bs4.element.ResultSet'>
120


In [4]:
%%time
# Find the total number of posts to find the limit of the pagination
city = 'Vancouver'
sub_city = 'van'
response = get(url_)
soup = BeautifulSoup(response.text, 'html.parser')
posts = soup.find_all('li', class_='result-row')
results_num = soup.find('div', class_='search-legend')
results_total = int(results_num.find('span', class_='totalcount').text)
print(f"Total number of search result: {results_total}")

# each page has 119 posts so each new page is defined as follows; s=120, s=240, .....
pages = np.arange(0, results_total+1, 120)

iterations = 0

post_date = []
post_cities = []
num_bedroom = []
sqfts = []
prices = []
post_titles = []
post_links = []

for page in pages:
    
    # get request
    url = Url(city_name=city, sub_city=sub_city, num_post=page)
    url_ = url.url
    response = get(url_)
    
    sleep(1)
    
    # throw warning for status code that are not 200
    if response.status_code != 200:
        warn(f'Request: {requests}; Status code: {response.status_code}')
        
    soup = BeautifulSoup(response.text, 'html.parser')
    
    posts = soup.find_all('li', class_='result-row')
    
    
    # Extract data 
    for post in posts:
        
        if post.find('span', class_ = 'result-hood') is not None:
            
            # Posting date
            post_datetime = post.find('time', class_='result-date')['datetime']
            post_date.append(post_datetime)
            
            # Neighbourhoods
            post_city = post.find('span', class_='result-hood').text.strip('( )')
            post_cities.append(post_city)
            
            # title text
            post_title = post.find('a', class_='result-title hdrlnk')
            post_title_text = post_title.text
            post_titles.append(post_title_text)
            
            # Post price in integer
            price = int(post.find('span', class_='result-price').text.split('$')[1].replace(',', ''))
            prices.append(price)
            
            # Post link
            post_link = post_title['href']
            post_links.append(post_link)
            
            if post.find('span', class_ = 'housing') is not None:
                
                #if the first element is accidentally square footage
                if 'ft2' in post.find('span', class_ = 'housing').text.split()[0]:
                    
                    #make bedroom nan
                    bedroom_count = np.nan
                    num_bedroom.append(bedroom_count)
                    
                    #make sqft the first element
                    sqft = int(post.find('span', class_ = 'housing').text.split()[0][:-3])
                    sqfts.append(sqft)
                    
                #if the length of the housing details element is more than 2
                elif len(post.find('span', class_ = 'housing').text.split()) > 2:
                    
                    #therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    num_bedroom.append(bedroom_count)
                    
                    #and sqft will be number 3, so set these here and append
                    sqft = int(post.find('span', class_ = 'housing').text.split()[2][:-3])
                    sqfts.append(sqft)
                    
                #if there is num bedrooms but no sqft
                elif len(post.find('span', class_ = 'housing').text.split()) == 2:
                    
                    #therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    num_bedroom.append(bedroom_count)
                    
                    #and sqft will be number 3, so set these here and append
                    sqft = np.nan
                    sqfts.append(sqft)                    
                
                else:
                    bedroom_count = np.nan
                    num_bedroom.append(bedroom_count)
                
                    sqft = np.nan
                    sqfts.append(sqft)
                
            #if none of those conditions catch, make bedroom nan, this won't be needed    
            else:
                bedroom_count = np.nan
                num_bedroom.append(bedroom_count)
                
                sqft = np.nan
                sqfts.append(sqft)
            #    num_bedroom.append(bedroom_count)
                
            #    sqft = np.nan
            #    sqfts.append(sqft)
        iterations += 1
        #print("Page" + str(iterations) + "Scraped Successfully!")   

Total number of search result: 3000
CPU times: total: 4.12 s
Wall time: 47 s


In [5]:
# create DataFrame
df = pd.DataFrame({'Post Datetime': post_date,
                   'City Code': city.upper()[:3],
                   'Area Code': sub_city.upper(),
                   'Post Title': post_titles,
                   'Post URL': post_links,
                  'Neighborhood': post_cities,
                  'Bedroom': num_bedroom,
                  'SQFT': sqfts,
                  'Price': prices})

print(df.info())

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3054 entries, 0 to 3053
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Post Datetime  3054 non-null   object 
 1   City Code      3054 non-null   object 
 2   Area Code      3054 non-null   object 
 3   Post Title     3054 non-null   object 
 4   Post URL       3054 non-null   object 
 5   Neighborhood   3054 non-null   object 
 6   Bedroom        2839 non-null   object 
 7   SQFT           2697 non-null   float64
 8   Price          3054 non-null   int64  
dtypes: float64(1), int64(1), object(7)
memory usage: 214.9+ KB
None


,Post Datetime,City Code,Area Code,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price
0,2022-12-14 11:02,VAN,VAN,get your free rent. ROOM DIVIDERS build & inst...,https://vancouver.craigslist.org/van/apa/d/van...,city of vancouver,3,NaN,1800
1,2022-12-14 11:00,VAN,VAN,"FULLY FURNISHED 1 Bed, 1 Bath, Den & Stand out...",https://vancouver.craigslist.org/van/apa/d/van...,Central Downtown Vancouver,1,600.0,2750
2,2022-12-14 10:59,VAN,VAN,FULLY FURNISHED Studio & Den with Private Balc...,https://vancouver.craigslist.org/van/apa/d/van...,Yaletown-Vancouver,NaN,500.0,2600
3,2022-12-14 10:58,VAN,VAN,"$500 Credit! Beautifully renovated 3BR/2BA, Pe...",https://vancouver.craigslist.org/van/apa/d/wes...,city of vancouver,3,1500.0,5249
4,2022-12-14 10:54,VAN,VAN,"LARGE 2 BEDROOM + DEN, 2 BATHROOMS, + PARKING",https://vancouver.craigslist.org/van/apa/d/van...,MILROSS,2,1287.0,4000



## Process


* Create and transform data. 
* Maintan data integrity
    * Validility 
    * Accuracy
    * Completeness
    * Consistency
* Test data
### Clean Data
    * Remove duplicate    
    * Find null values then decide how to fill or drop entire row
    * Sorting
    * Filtering
Verify and report on cleaning results

Check the dataset I created from the Craigslist ad and final adjustment of the dataset to analyze the data. 

First to check the dataset:

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3054 entries, 0 to 3053
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Post Datetime  3054 non-null   object 
 1   City Code      3054 non-null   object 
 2   Area Code      3054 non-null   object 
 3   Post Title     3054 non-null   object 
 4   Post URL       3054 non-null   object 
 5   Neighborhood   3054 non-null   object 
 6   Bedroom        2839 non-null   object 
 7   SQFT           2697 non-null   float64
 8   Price          3054 non-null   int64  
dtypes: float64(1), int64(1), object(7)
memory usage: 214.9+ KB


Convert Post Datetime object to Datetime.

Connect City Code and Area Code then create Post Area Code. (In this DataFrame, it only contain one area of data but I am storeing 13 different area in CSV file. This code will distinguish the data in different regions.)

I would like to distingish FULLY FURNISHED suite and not furnished suite. I will create another column to show if it isTURE or FALSE. I noticed the title says if it is furnished suite or not (If not on the title, I assume that the suite is not furnished) 

Also, I noticed that the number of bedroom is string, so I will convert to integer. 

Sort DataFrame Descending order by Post Datetime.

Remove duplicate. 

Is there any oulier? 

In [7]:
# Convert Datetime then split seperate columns
df['Post Datetime'] = pd.to_datetime(df["Post Datetime"])
df['Post Date'] = df['Post Datetime'].dt.date
df['Post Time'] = df['Post Datetime'].dt.time

df["PostAreaCode"] = df["City Code"] + df["Area Code"]

df = df[["PostAreaCode", 'Post Datetime', 'Post Date', 'Post Time', 'Post Title', 'Post URL', 'Neighborhood', 'Bedroom', 'SQFT', 'Price']]
df.head()

,PostAreaCode,Post Datetime,Post Date,Post Time,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price
0,VANVAN,2022-12-14 11:02:00,2022-12-14,11:02:00,get your free rent. ROOM DIVIDERS build & inst...,https://vancouver.craigslist.org/van/apa/d/van...,city of vancouver,3,NaN,1800
1,VANVAN,2022-12-14 11:00:00,2022-12-14,11:00:00,"FULLY FURNISHED 1 Bed, 1 Bath, Den & Stand out...",https://vancouver.craigslist.org/van/apa/d/van...,Central Downtown Vancouver,1,600.0,2750
2,VANVAN,2022-12-14 10:59:00,2022-12-14,10:59:00,FULLY FURNISHED Studio & Den with Private Balc...,https://vancouver.craigslist.org/van/apa/d/van...,Yaletown-Vancouver,NaN,500.0,2600
3,VANVAN,2022-12-14 10:58:00,2022-12-14,10:58:00,"$500 Credit! Beautifully renovated 3BR/2BA, Pe...",https://vancouver.craigslist.org/van/apa/d/wes...,city of vancouver,3,1500.0,5249
4,VANVAN,2022-12-14 10:54:00,2022-12-14,10:54:00,"LARGE 2 BEDROOM + DEN, 2 BATHROOMS, + PARKING",https://vancouver.craigslist.org/van/apa/d/van...,MILROSS,2,1287.0,4000


In [8]:
# Convert Bedroom number strings to Integer then NaN value to 0 assume 0 bedroom is studio appartment. 
df["Bedroom"] = pd.to_numeric(df["Bedroom"], errors='ignore').astype('Int64')
df["Bedroom"] = df["Bedroom"].fillna(0)

In [9]:
%%time
# Check if word 'furnished' in the title post. 
search_word = 'furnished'
furnished = []
for i in range(len(df)):
    post_title_list = df['Post Title'][i].lower().split(" ")
    if any(word == search_word for word in post_title_list):
        furnished.append(True)
    else:
        furnished.append(False)
df["Furnished?"] = furnished

CPU times: total: 0 ns
Wall time: 15.1 ms


In [10]:
# Sort DataFrame by Datetime
df.sort_values(by="Post Datetime", axis=0, ascending=False, inplace=True)

In [11]:
# Remove duplicates
df.drop_duplicates(inplace=True)
len(df)

3000

In [12]:
# Find out outlier -Post Price between $1 to $300
df[df["Price"].between(1, 300)]

,PostAreaCode,Post Datetime,Post Date,Post Time,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price,Furnished?
1120,VANVAN,2022-12-12 13:20:00,2022-12-12,13:20:00,Vancouver Furnished Master Bedroom for Rent,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,1,400.0,120,True
2175,VANVAN,2022-12-06 21:27:00,2022-12-06,21:27:00,Sublet available Vancouver Dec 19th to Jan 2nd,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,1,NaN,90,False
2535,VANVAN,2022-12-04 15:12:00,2022-12-04,15:12:00,DT One Bedroom One Bath Furnished Dec 21-29 2022,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,1,650.0,100,True
2791,VANVAN,2022-11-30 20:47:00,2022-11-30,20:47:00,1 Bedroom suite furnished,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver East,1,450.0,150,True
2858,VANVAN,2022-11-29 22:17:00,2022-11-29,22:17:00,Bright Apartment to rent,https://vancouver.craigslist.org/van/apa/d/van...,Kitsilano,1,NaN,100,False
2899,VANVAN,2022-11-29 10:38:00,2022-11-29,10:38:00,1BR Apartment,https://vancouver.craigslist.org/van/apa/d/van...,city of vancouver,1,628.0,1,False


In [13]:
# One of the low price post (post Price $120)
df.loc[822]["Post URL"]

'https://vancouver.craigslist.org/van/apa/d/vancouver-luxury-bdrm-den-condo-at/7565720525.html'

This post is daily rental. From this, I assume teh rental price around $100 would be daily rental suite. 

What about $1 or $0 posts?

In [14]:
# $1 ads
one_dollar = df[df["Price"] == 1]
one_dollar["Post URL"]

2899    https://vancouver.craigslist.org/van/apa/d/van...
Name: Post URL, dtype: object

In [16]:
# $0 ads
zero_dollar = df[df["Price"] == 0]
zero_dollar

,PostAreaCode,Post Datetime,Post Date,Post Time,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price,Furnished?
30,VANVAN,2022-12-14 10:31:00,2022-12-14,10:31:00,"SCAM $2,045 / 1br - 635ft2 - Suite 1 BR/Quiet ...",https://vancouver.craigslist.org/van/apa/d/van...,kits,1,NaN,0,False
182,VANVAN,2022-12-14 08:12:00,2022-12-14,08:12:00,Westsea Place - (Westend) - Studio & 1 Bedroom...,https://vancouver.craigslist.org/van/apa/d/del...,1644 NELSON ST.,0,NaN,0,False
446,VANVAN,2022-12-13 15:07:00,2022-12-13,15:07:00,CREEP ALERT,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,3,NaN,0,False
579,VANVAN,2022-12-13 11:30:00,2022-12-13,11:30:00,DO NOT LIVE HERE - 124 DUNLEVY AVE,https://vancouver.craigslist.org/van/apa/d/van...,city of vancouver,0,NaN,0,False
714,VANVAN,2022-12-13 08:23:00,2022-12-13,08:23:00,Westsea Place - (Westend) - Studio & 1 Bedroom...,https://vancouver.craigslist.org/van/apa/d/del...,1644 NELSON ST.,0,NaN,0,False
1289,VANVAN,2022-12-12 09:19:00,2022-12-12,09:19:00,Westsea Place - (Westend) - Studio & 1 Bedroom...,https://vancouver.craigslist.org/van/apa/d/van...,1644 NELSON ST.,0,NaN,0,False
1950,VANVAN,2022-12-08 14:02:00,2022-12-08,14:02:00,DO NOT RENT 1310 WEST 10TH AVE,https://vancouver.craigslist.org/van/apa/d/van...,Fairview,1,NaN,0,False
2277,VANVAN,2022-12-06 11:56:00,2022-12-06,11:56:00,Nice Affordable Comfy and Cozy Scams,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,0,NaN,0,False
2302,VANVAN,2022-12-06 01:57:00,2022-12-06,01:57:00,No Cooking is Illegal,https://vancouver.craigslist.org/van/apa/d/van...,city of vancouver,1,NaN,0,False
2924,VANVAN,2022-11-28 19:50:00,2022-11-28,19:50:00,"SCAM -$1,398 / 1br - Quiet, peaceful 1 BDRM & DEN",https://vancouver.craigslist.org/van/apa/d/van...,"4515 Langara Avenue Vancouver, BC,",0,NaN,0,False


I think it most likely $0 and $1 ads are just not showing the price, so I will drop those rows from DataFrame. 

Tne price around $100 is most likely daily rental so I will store in different variable. 

In [39]:
day_rental = df[df["Price"].between(10, 300)]
day_rental

,PostAreaCode,Post Datetime,Post Date,Post Time,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price,Furnished?,Weekday
1120,VANVAN,2022-12-12 13:20:00,2022-12-12,13:20:00,Vancouver Furnished Master Bedroom for Rent,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,1,400.0,120,True,1
2175,VANVAN,2022-12-06 21:27:00,2022-12-06,21:27:00,Sublet available Vancouver Dec 19th to Jan 2nd,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,1,NaN,90,False,2
2535,VANVAN,2022-12-04 15:12:00,2022-12-04,15:12:00,DT One Bedroom One Bath Furnished Dec 21-29 2022,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,1,650.0,100,True,7
2791,VANVAN,2022-11-30 20:47:00,2022-11-30,20:47:00,1 Bedroom suite furnished,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver East,1,450.0,150,True,3
2858,VANVAN,2022-11-29 22:17:00,2022-11-29,22:17:00,Bright Apartment to rent,https://vancouver.craigslist.org/van/apa/d/van...,Kitsilano,1,NaN,100,False,2


In [18]:
# Drop low-end outliers rows
drop_index = []
for idx in one_dollar.index:
    drop_index.append(idx)
for idx in zero_dollar.index:
    drop_index.append(idx)
print(f"Original DataFrame Size:{len(df)}\nNumber of Dropped Rows: {len(drop_index)}")
df.drop(drop_index,axis=0, inplace=True)
print(f"DataFrame size after drop rows: {len(df)}")

Original DataFrame Size:3000
Number of Dropped Rows: 13
DataFrame size after drop rows: 2987


What about high-end outliers?

In [42]:
# 20 higest price 
df.sort_values(by="Price", ascending=False).head(20)

,PostAreaCode,Post Datetime,Post Date,Post Time,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price,Furnished?,Weekday
2355,VANVAN,2022-12-05 20:16:00,2022-12-05,20:16:00,"Fully Furnished 2bdrm, 2 bath condo for 1 month",https://vancouver.craigslist.org/van/apa/d/van...,Downtown Vancouver,2,800.0,500000,True,1
3041,VANVAN,2022-11-25 16:48:00,2022-11-25,16:48:00,Modern Pet friendly House at 4305 Locarno Cres...,https://vancouver.craigslist.org/van/apa/d/van...,West Point Grey,4,3958.0,25000,False,5
1255,VANVAN,2022-12-12 10:16:00,2022-12-12,10:16:00,Furnished Two Bedroom Sub-Penthouse at Shangri...,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,2,2694.0,25000,True,1
1231,VANVAN,2022-12-12 10:56:00,2022-12-12,10:56:00,Ultra Customized Morden Luxury House In West P...,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,4,3958.0,25000,False,1
2052,VANVAN,2022-12-07 15:07:00,2022-12-07,15:07:00,Furnished Two Bedroom Sub-Penthouse at Shangri...,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,2,2694.0,25000,True,3
369,VANVAN,2022-12-13 17:48:00,2022-12-13,17:48:00,Luxury Furnished 3 Bed + Den 4 Bath Condo at T...,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,3,2223.0,24995,True,2
228,VANVAN,2022-12-14 01:08:00,2022-12-14,01:08:00,Furnished luxury Mansion 7 bed/7 bath availabl...,https://vancouver.craigslist.org/van/apa/d/wes...,west vancouver,6,10000.0,22880,True,3
703,VANVAN,2022-12-13 08:57:00,2022-12-13,08:57:00,Coal Harbour Luxury Three Bedroom Apartment,https://vancouver.craigslist.org/van/apa/d/van...,Vancouver,3,3636.0,20000,False,2
2445,VANVAN,2022-12-05 09:59:00,2022-12-05,09:59:00,Luxurious 3 Bedroom Primary Penthouse at the A...,https://vancouver.craigslist.org/van/apa/d/van...,Yaletown,3,2223.0,20000,False,1
1076,VANVAN,2022-12-12 14:50:00,2022-12-12,14:50:00,One of a Kind: 4BR Luxury Unit@Hotel Georgia: ...,https://vancouver.craigslist.org/van/apa/d/van...,Dowtown,4,3700.0,19998,False,1


In [60]:
# Rental price higher than $5000.
more_than_8000 = df[df["Price"] > 000]
more_than_8000.groupby(by=["Bedroom", "Furnished?"]).agg({'SQFT': ["min", "max"],
                                                         "Price": ['min', 'max', 'count']})

SQFT           Price              
                       min      max    min     max count
Bedroom Furnished?                                      
0       False        103.0   1200.0    550    4050   142
        True         180.0   1357.0   1000    8000    63
1       False          1.0   2000.0     90    6500   736
        True         361.0   1855.0    100   10000   304
2       False        500.0   2936.0   1000   18000   958
        True         512.0   2873.0   1175  500000   286
3       False          1.0   4000.0   1250   20000   263
        True         781.0   4000.0   2500   24995    54
4       False        900.0   4600.0    950   25000    85
        True        2000.0   6800.0   4800   18500    15
5       False       1500.0   6426.0   3200   19500    37
        True        2250.0   6500.0   6088   14995     9
6       False       2300.0  10098.0    950   16500    18
        True        4639.0  29789.0   5600   22880     6
7       False       1300.0   4800.0   1300    9995     6
        True        6000.0   6000.0  19995   19995     1
8       False       4980.0   4980.0   6500    7900     4

In [52]:
# What kind of 2 bedroom suite charges $1800??
idx_list = df[df["Price"] == 18000].index
df.loc[idx_list[0]]["Post URL"]

'https://vancouver.craigslist.org/van/apa/d/vancouver-vancouver-downtown-fairmont/7554964282.html'

In [59]:
# Most expensive rental in Vancouver
most_expensive_idx = df.sort_values("Price", ascending=False).head(1).index
df.loc[most_expensive_idx[0]]["Post URL"]

'https://vancouver.craigslist.org/van/apa/d/vancouver-fully-furnished-2bdrm-bath/7557989366.html'

From above data, Downtown Vanvouver rental market is different world. I must make decision to define what is the outlier of high-end rental. 

If the number of the high-end prices are large enough, I can make DataFrame for only high-end and search for the high-end market opportunities. 

Clearly, I should not mix with regular market and high-end market because it will provide me biased price average to determin the valuse of the house. 

I am wondering that <a href='https://www03.cmhc-schl.gc.ca/hmip-pimh/en/TableMapChart/Table?TableId=2.1.31.3&GeographyId=2410&GeographyTypeId=3&DisplayAs=Table&GeograghyName=Vancouver'>CMHC Vancouver rental market report</a> concerns about those outliers. 

## Analyze

I will use Pandas to analyze the data. 

#### First, what data do I want to know?
* What is the percentage of the Furnished rental suite? 
* Is there any price different? 
* How many bedroom suite has most on ads? And is it any different in area?
* How much is the rental price in general? 


In [19]:
# Count how many rentals are furnished 
num_furnished_suite = df[df['Furnished?'] == True]["Furnished?"].count()
num_unfurnished_suite = len(df) - num_furnished_suite
furnished_suite_percent = np.round((num_furnished_suite / len(df)) * 100, 2)
print(f"Number of furnished suite is {num_furnished_suite}, and it's {furnished_suite_percent}% of the entire list.")

Number of furnished suite is 738, and it's 24.71% of the entire list.


In [20]:
# Find out the number of ads on each bedrooms number.  
num_bed = np.sort(df["Bedroom"].unique())
suite_counts = []
for i in range(len(num_bed)):
    suite_count = len(df[df['Bedroom'] == num_bed[i]])
    suite_counts.append(suite_count)
    #print(f'Number of {num_bed[i]} Bedroom suite: {len(df[df["Bedroom"] == num_bed[i]])}')
#print(f"List of suite counts: {suite_counts}")

In [21]:
# Find out Mean, Max, Min of SQFT and Price of each bedroom and SUM of furnished suite and total suite on each bedroom. 
df_count = df.groupby(by=['Bedroom']).agg({'SQFT': ['mean', 'max', 'min'],
                                                 'Price': ['mean', 'max', 'min'],
                                                  'Furnished?': ['sum', 'count']})
df_count['percent furnished'] = df_count['Furnished?']['sum'] / df_count["Furnished?"]['count']
df_count

SQFT                         Price               Furnished?  \
                mean      max     min         mean     max   min        sum   
Bedroom                                                                       
0         433.873626   1357.0   103.0  2281.160976    8000   550         63   
1         614.444196   2000.0     1.0  2561.443269   10000    90        304   
2         986.626667   2936.0   500.0  4333.052251  500000  1000        286   
3        1538.791096   4000.0     1.0  5444.517350   24995  1250         54   
4        2598.761364   6800.0   900.0  6534.730000   25000   950         15   
5        3539.175000   6500.0  1500.0  8095.695652   19500  3200          9   
6        6389.045455  29789.0  2300.0  9554.375000   22880   950          6   
7        4028.333333   6000.0  1300.0  9084.285714   19995  1300          1   
8        4980.000000   4980.0  4980.0  7550.000000    7900  6500          0   

              percent furnished  
        count                    
Bedroom                          
0         205          0.307317  
1        1040          0.292308  
2        1244          0.229904  
3         317          0.170347  
4         100          0.150000  
5          46          0.195652  
6          24          0.250000  
7           7          0.142857  
8           4          0.000000

In [22]:
# Price per furnished suit on each bedroom
#df_price_furnished = df.drop(['Post Datetime', 'Post Title', 'Post URL', 'Neighborhood'], axis=1)
df_price_furnished = df.groupby(['Bedroom', 'Furnished?']).agg({'Price': ['mean', 'max', 'min', 'count'],
                                                               "SQFT": ['mean', 'max', 'min']})
df_price_furnished

Price                               SQFT           \
                            mean     max    min count          mean      max   
Bedroom Furnished?                                                             
0       False        2217.880282    4050    550   142    438.336000   1200.0   
        True         2423.793651    8000   1000    63    424.087719   1357.0   
1       False        2410.391304    6500     90   736    615.590909   2000.0   
        True         2927.148026   10000    100   304    611.921429   1855.0   
2       False        3719.717119   18000   1000   958    973.585480   2936.0   
        True         6387.510490  500000   1175   286   1027.723247   2873.0   
3       False        5070.539924   20000   1250   263   1513.531381   4000.0   
        True         7265.925926   24995   2500    54   1652.698113   4000.0   
4       False        6061.623529   25000    950    85   2536.324324   4600.0   
        True         9215.666667   18500   4800    15   2928.785714   6800.0   
5       False        7776.459459   19500   3200    37   3498.225806   6426.0   
        True         9408.111111   14995   6088     9   3680.222222   6500.0   
6       False        8656.944444   16500    950    18   4907.705882  10098.0   
        True        12246.666667   22880   5600     6  11425.600000  29789.0   
7       False        7265.833333    9995   1300     6   3634.000000   4800.0   
        True        19995.000000   19995  19995     1   6000.000000   6000.0   
8       False        7550.000000    7900   6500     4   4980.000000   4980.0   

                            
                       min  
Bedroom Furnished?          
0       False        103.0  
        True         180.0  
1       False          1.0  
        True         361.0  
2       False        500.0  
        True         512.0  
3       False          1.0  
        True         781.0  
4       False        900.0  
        True        2000.0  
5       False       1500.0  
        True        2250.0  
6       False       2300.0  
        True        4639.0  
7       False       1300.0  
        True        6000.0  
8       False       4980.0

Dropped extrimely low price add, and it seems no outlier on the price

In [36]:
# Count how many ads per day.1=Mon, 2=Tue, ......, 7=Sun
df["Weekday"] = df["Post Datetime"].dt.isocalendar().day
num_ad_per_day_df = df.groupby('Post Date').agg({'Post Title': 'count',
                                                "Weekday": 'first'})

num_ad_per_day_df

,Post Title,Weekday
Post Date,,
2022-11-21,1,1
2022-11-24,1,4
2022-11-25,24,5
2022-11-26,19,6
2022-11-27,43,7
2022-11-28,48,1
2022-11-29,56,2
2022-11-30,73,3
2022-12-01,88,4


Which day of the week have more ads? 

Is there any day of the week that having more ads than others? 

## Share

#### Data Visualization:


In [63]:
import matplotlib.pyplot as plt
import seaborn as sns

## Act